## IBSI Chapter 1 Phase 1 − Radiomic Computations

@Author : [MEDomics consortium](https://github.com/medomics/)

@EMAIL : medomics.info@gmail.com

@REF : [IBSI 1](https://arxiv.org/pdf/1612.07003.pdf)

**STATEMENT**:
This file is part of <https://github.com/MEDomics/MEDomicsLab/>,
a package providing PYTHON programming tools for radiomics analysis.
--> Copyright (C) MEDomicsLab consortium.

This package is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This package is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this package.  If not, see <http://www.gnu.org/licenses/>.

### Introduction


In this notebook we treat the first phase of standardization of image processing and feature computation. In the figure below, we focus on the first part, phase 1. So we only compute radiomics features from a digital phantom without any processing
<img src="https://www.researchgate.net/profile/Emiliano-Spezi/publication/339835844/figure/fig1/AS:867857227337731@1583924695905/Flowchart-of-study-overview-The-workflow-in-a-typical-radiomics-analysis-starts-with.ppm" alt="Flowchart of radiomics study" style="width:500px;"/>

### Dataset - Digital phantom
In this chapter and in this phase, reference values for features were obtained using a digital image phantom, which is described below. The digital phantom can be found here: https://github.com/theibsi/data_sets/tree/master/ibsi_1_digital_phantom

- The phantom consists of 5 × 4 × 4 (x, y, z) voxels.
- A slice consists of the voxels in (x, y) plane for a particular slice at position z. Slices are therefore stacked in the z direction.
- Voxels are 2.0 × 2.0 × 2.0 mm in size.
- Not all voxels are included in the region of interest. Several excluded voxels are located on the outside of the ROI, and one internal voxel was excluded as well. Voxels excluded from the ROI are shown in blue in figure below.
- Some intensities are not present in the phantom. Notably, grey levels 2 and 5 are absent. 1 is the lowest grey level present in the ROI, and 6 the highest.

<img src="https://www.researchgate.net/profile/Alex-Zwanenburg/publication/311805734/figure/fig4/AS:867063404965890@1583735433294/Exploded-view-of-the-test-volume-The-number-in-each-voxel-corresponds-with-its-grey.png" alt="IBSI 1 Phase 1 Digital Phantom" style="width:400px;"/>

In [ ]:
import argparse
import json
import os
import pickle
import sys

MODULE_DIR = os.path.dirname(os.path.abspath('../MEDimage/MEDimage.py'))
sys.path.append(os.path.dirname(MODULE_DIR))

from copy import deepcopy
from itertools import product
from pathlib import Path

from numpyencoder import NumpyEncoder

from MEDimage.MEDimage import MEDimage
from MEDimage.MEDimageComputeRadiomics import MEDimageComputeRadiomics
from MEDimage.MEDimageProcessing import MEDimageProcessing
from MEDimage.utils import jsonUtils

In [ ]:
from MEDimage.processing.getROI_fromIndexes import getROI_fromIndexes
from MEDimage.processing.roiExtract import roiExtract

In [ ]:
def __getPathResults():
    if not (Path(os.getcwd()) / "results/ibsi1/phase1").exists():
        _rp = Path(os.getcwd()) / "results/ibsi1/phase1"
        Path.mkdir(_rp, parents=True)
    return _rp

### Initialization

We start by initializing the important paths (settings folder, dataset folder...). Then the important variables: file name, ROI name and load the computation parameters.

In [ ]:
pathData = Path(os.getcwd()) / "data" # Path to the digital phantom folder
pathSettings = Path(os.getcwd()) / "settings" # Path to the script settings/configuration folder
pathRead = pathData /'Phantom' # Path to the digital phantom
nameRead = 'FIG61__SEG(tumorAuto).CTscan.nii.gz' # Digital phantom file name
nameROI = '{tumor}' # Region of interest name
roiType = ''
roiType_label = ''
# Load script parameters
imParams = jsonUtils.loadjson(pathSettings / 'Phase1_settings.json')

In [ ]:
def init_MEDimage():
    """
    Initializes the MEDimage class and the child classes.
    :return: Child classes (MEDimageProcessing and MEDimageComputeRadiomics).
    """ 

    if nameRead.endswith('.npy'):
        # MEDimage instance is now in Workspace
        with open(pathRead / nameRead, 'rb') as f: MEDimg = pickle.load(f)

        # MEDimageProcess instance is now in Workspace
        MEDimageProcess = MEDimageProcessing(MEDimg=MEDimg, log_file=pathRead /'log_file_ibsi1p1.txt')

        # MEDimageComputeRadiomics instance is now in Workspace
        MEDimageCR = MEDimageComputeRadiomics(MEDimg=MEDimg, log_file=MEDimageProcess.log_file)

        # Initialize processing & computation parameters
        MEDimageProcess.init_Params(imParamScan=imParams,
                                    imParamFilter=imParams['imParamFilter'],
                                    roiType=roiType)

        # Clear MEDimage instance # Clear up RAM
        MEDimg = None 

        return MEDimageProcess, MEDimageCR

    # Set up NIFTI Image path 
    NiftiImage = pathRead / nameRead

    # MEDimage instance is now in Workspace
    MEDimg = MEDimage()

    # Initialization using NIFTI file :
    MEDimg.init_from_nifti(NiftiImagePath=NiftiImage)

    # spatialRef Creation : 
    MEDimg.scan.volume.spatialRef_from_NIFTI(NiftiImage)

    # MEDimageProcess instance is now in Workspace
    MEDimageProcess = MEDimageProcessing(MEDimg=MEDimg, log_file=pathRead /'log_file_ibsi1p1.txt')

    # MEDimageComputeRadiomics instance is now in Workspace
    MEDimageCR = MEDimageComputeRadiomics(MEDimg=MEDimg, log_file=MEDimageProcess.log_file)

    # Initialize processing & computation parameters
    MEDimageProcess.init_Params(imParamScan=imParams,
                                imParamFilter=imParams['imParamFilter'],
                                roiType=roiType)

    # Clear MEDimage instance, clear up RAM
    MEDimg = None

    return MEDimageProcess, MEDimageCR

In [ ]:
def process_image(MEDimageProcess, MEDimageCR):
    """
    Extracts ROI mask and creates intensity mask.
    :param MEDimgProcess: Instance of MEDImageProcessing.
    :param MEDImageCR: Instance of MEDImageComputeRadiomics.
    :return: Four image volume objects (Intensity mask, morphological mask and their volume object).
    """
    # Extraction of ROI mask :
    volObjInit, roiObjInit = getROI_fromIndexes(MEDimageProcess, nameROI=nameROI,boxString='full')
 
    # Morphological Mask :
    volObj = deepcopy(volObjInit)
    roiObj_Morph = deepcopy(roiObjInit)

    # Intensity Mask Creation :
    roiObj_Int = deepcopy(roiObj_Morph)

    # Preparation of computation :
    MEDimageCR.init_NTF_Calculation(volObj)

    # Image volume ROI Extraction :
    volInt_RE = roiExtract(
        vol=volObj.data, 
        roi=roiObj_Int.data
    )

    return volObj, roiObj_Morph, volInt_RE, roiObj_Int

In [ ]:
def save_results(MEDImageCR, pathResults):
    """
    Saves the results in a JSON file under the name : Results_P1.json
    :param MEDImageCR: Instance of MEDImageComputeRadiomics class.
    :return: None.
    """
    with open(pathResults / f"phase1/Results_P1.json", "w") as fp:   
        json.dump(MEDImageCR.Params['radiomics']['image'], fp, indent=4, cls=NumpyEncoder)

    return 0

### Classes initilization
In the IBSI scripts we are going to use the **MEDimage** class and its derived classes (**MEDimageProcessing** and **MEDimageComputeRadiomics**) to process the images and to extract the features. 
- **MEDimage**: Is a Python class that organizes all scan data and many other useful information that is used by the many processing and computing methods.
- **MEDimageProcessing**: A **MEDimage** derived class that uses the inherited attributes and many other non-inherited attributes to process the imaging data (interpolation, segmentation...).
- **MEDimageComputeRadiomics**: Another **MEDimage** derived class that uses the inherited attributes and many other non-inherited attributes to compute/extract features (statistical features, morphological features...) from the imaging data.


So the first step is to initialize the MEDimage class either using a **NIFTI** or a **NPY** file. The npy format already contains a MEDimage instance, for the NIFTI format make sure the mask file is in the same folder with the correct name with the following conventions: 
- NPY format: **PatientID__ImagingScanName.ImagingModality.npy**
- NIfTI format: 
    - Tumor volume: **PatientID__ImagingScanName(tumorAuto).ImagingModality.nii.gz**
    - ROI mask: **PatientID__ImagingScanName(tumor).ROI.nii.gz**

In [ ]:
MEDimageProcess, MEDimageCR = init_MEDimage() 

### Image processing
For this phase the only processing we need is to extract the ROI and replace the excluded values (values outside the ROI) in the image volume with a placeholder (NaN). The intensity and morphological mask are identical in this case (No re-segmentation is done here).

In [ ]:
volObj, roiObj_Morph, volInt_RE, roiObj_Int = process_image(MEDimageProcess, MEDimageCR)

### Non-Texture Features extraction
In this section we extract the following famillies of features using MEDimageComputeRadiomics methods : 

*morphological features, local intensity, statistical, Intensity-based and intensity histogram-based.*

No further image processing is required.

#### Morphological features

Morphological features describe geometric aspects of a region of interest (ROI), such as area and
volume. Morphological features are based on ROI voxel representations of the volume.

In [ ]:
from MEDimage.biomarkers.getMorphFeatures import getMorphFeatures

MORPH = getMorphFeatures(
    vol=volObj.data, 
    maskInt=roiObj_Int.data, 
    maskMorph=roiObj_Morph.data,
    res=MEDimageCR.Params['scaleNonText'],
    intensity=MEDimageCR.Params['intensity']
)

#### Local intensity features

Voxel intensities within a defined neighborhood around a center voxel are used to compute local
intensity features. By definition, local intensity features are calculated in 3D, and not per slice.

In [ ]:
from MEDimage.biomarkers.getLocIntFeatures import getLocIntFeatures

LocalIntensity = getLocIntFeatures(
    imgObj=volObj.data, 
    roiObj=roiObj_Int.data,
    res=MEDimageCR.Params['scaleNonText'],
    intensity=MEDimageCR.Params['intensity']
)

#### Intensity-based statistical features

The intensity-based statistical features describe how intensities within the region of interest (ROI)
are distributed. The features in this set do not require discretization, and may be used to describe
a continuous intensity distribution.

In [ ]:
from MEDimage.biomarkers.getStatsFeatures import getStatsFeatures

Stats = getStatsFeatures(
    vol=volInt_RE,
    intensity=MEDimageCR.Params['intensity']
)

#### Intensity histogram features

An intensity histogram is generated by discretizing the original intensity distribution into
intensity bins.

In [ ]:
from MEDimage.biomarkers.getIntHistFeatures import getIntHistFeatures

IntHistFeatures = getIntHistFeatures(
    vol=volInt_RE
)

#### Intensity-volume histogram features

The (cumulative) intensity-volume histogram (IVH) of the set of voxel intensities in the ROI
intensity mask describes the relationship between discretized intensity and the fraction of the
volume containing at least intensity the same intensity.

In [ ]:
from MEDimage.biomarkers.getIntVolHistFeatures import getIntVolHistFeatures

IntensityVolHistogram = getIntVolHistFeatures(
            MEDimg=MEDimageProcess,
            vol=volInt_RE,
            volInt_RE=volInt_RE, 
            wd=1
)

### Texture Features extraction
In this section, for each text scale<sup>1</sup> we extract the matrix-based features using MEDimageComputeRadiomics methods : 

*Grey level co-occurrence based features (GLCM), grey level run length based features (GLRLM), grey level size zone matrix (GLSZM), grey level distance zone matrix (GLDZM), neighborhood grey tone difference matrix (NGTDM) and neighboring grey level dependence matrix (NGLDM).*

After the computation is finished, we update the radiomics structure (update the attributes for results). 

No further image processing is done in this section as well.

<sup>1</sup> For each time we resample the voxel spacing (In this case we resample the voxel spacing one time).

**Note**: For our case (IBSI 1, Phase 1) we only re-sample the voxel spacing one time so the texture features will be calculated one time.

In [ ]:
# Intensity mask creation :
roiObj_Int = deepcopy(roiObj_Morph)

# Preparation of computation :
MEDimageCR.init_TF_Calculation(Algo=0, Gl=0, Scale=0)

# ROI Extraction :
volQuant_RE = roiExtract(
    vol=volObj.data, 
    roi=roiObj_Int.data
    )

#### Grey level co-occurrence based features

The grey level co-occurrence matrix (GLCM) is a matrix that expresses how combinations of
discretized intensities (grey levels) of neighboring pixels, or voxels in a 3D volume, are distributed
along one of the image directions.

In [ ]:
from MEDimage.biomarkers.getGLCMfeatures import getGLCMfeatures

GLCM = getGLCMfeatures(
        vol=volQuant_RE, 
        distCorrection=MEDimageCR.Params['radiomics']['imParam']['image']['glcm']['distCorrection'])

#### Grey level run length based features
The grey level run length matrix (GLRLM) defines various texture features. Like the grey level co-occurrence matrix, GLRLM also assesses the distribution of
discretized grey levels in an image or in a stack of images. However, whereas GLCM assesses
co-occurrence of grey levels within neighboring pixels or voxels, GLRLM assesses run lengths. A
run length is defined as the length of a consecutive sequence of pixels or voxels with the same grey
level along direction.

In [ ]:
from MEDimage.biomarkers.getGLRLMfeatures import getGLRLMfeatures

GLRLM = getGLRLMfeatures(
        vol=volQuant_RE, 
        distCorrection=MEDimageCR.Params['radiomics']['imParam']['image']['glrlm']['distCorrection'])

#### Grey level size zone based features

The grey level size zone matrix (GLSZM) counts the number of groups (or zones) of linked voxels.
Voxels are linked if the neighboring voxel has an identical neighboring grey level.

In [ ]:
from MEDimage.biomarkers.getGLSZMfeatures import getGLSZMfeatures

GLSZM = getGLSZMfeatures(
        vol=volQuant_RE
        )

#### Grey level distance zone based features

The grey level distance zone matrix (GLDZM) counts the number of groups (or zones) of linked
voxels which share a specific neighboring grey level value and possess the same distance to ROI
edge. The GLDZM thus captures the relation between location and grey level.

In [ ]:
from MEDimage.biomarkers.getGLDZMfeatures import getGLDZMfeatures

GLDZM = getGLDZMfeatures(
        volInt=volQuant_RE, 
        maskMorph=roiObj_Morph.data
        )

####  Neighbourhood grey tone difference based features

The neighborhood grey tone difference matrix (NGTDM) contains the sum of grey level differences
of pixels/voxels with a discretized grey level and the average discretized grey level of neighboring pixels/voxels within a Chebyshev distance. 

In [ ]:
from MEDimage.biomarkers.getNGTDMfeatures import getNGTDMfeatures

NGTDM = getNGTDMfeatures(
        vol=volQuant_RE, 
        distCorrection=MEDimageCR.Params['radiomics']['imParam']['image']['ngtdm']['distCorrection'])

####  Neighbouring grey level dependence based features:

The neighbouring grey level dependence matrix (NGLDM) aims to capture the coarseness of the overall
texture and is rotationally invariant.

In [ ]:
from MEDimage.biomarkers.getNGLDMfeatures import getNGLDMfeatures

NGLDM = getNGLDMfeatures(
        vol=volQuant_RE
        )

In [ ]:
# Updating radiomics structure
MEDimageCR.updateRadiomics(IntVolHistFeatures=IntensityVolHistogram, MORPHFeatures=MORPH,
            LocalIntensityFeatures=LocalIntensity, StatsFeatures=Stats, IntHistFeatures=IntHistFeatures,
            GLCMFeatures=GLCM, GLRLMFeatures=GLRLM, GLSZMFeatures=GLSZM, GLDZMFeatures=GLDZM, 
            NGTDMFeatures=NGTDM, NGLDMFeatures=NGLDM)


Finally we print the results

In [ ]:
print(json.dumps(MEDimageCR.Params['radiomics']['image'], indent=4, cls=NumpyEncoder))

Run this cell to save the results in JSON format on your machine

In [ ]:
#pathResults = __getPathResults() # Path to where the results are gonna be saved 
#save_results(MEDImageCR, pathResults)